<a href="https://colab.research.google.com/github/MdUmairZafar/lab-2/blob/master/transferlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
import zipfile
import os
from PIL import Image  # Correctly import Image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

from deep_sort.deep_sort import DeepSort
from yolov4.tf import YOLOv4

# Mount Google Drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:

zip_path = '/content/drive/MyDrive/Dataset1.zip'
extract_path = '/content/Dataset1'
dataset_path = os.path.join(extract_path, 'train')

# Extract Dataset
if not os.path.exists(extract_path):
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_path)
    print(f"Dataset extracted to {extract_path}")
else:
    print(f"Dataset already exists at {extract_path}")


Dataset extracted to /content/Dataset1


In [ ]:

# Function to clean dataset
def clean_dataset(dataset_path):
    for root, dirs, files in os.walk(dataset_path):
        for file in files:
            file_path = os.path.join(root, file)
            try:
                # Attempt to open the file as an image
                with Image.open(file_path) as img:
                    img.verify()  # Verify if it's a valid image
            except (IOError, SyntaxError):
                print(f"Removing invalid file: {file_path}")
                os.remove(file_path)

# Clean the dataset
clean_dataset('/content/Dataset1')

# Data Preparation
datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

train_data = datagen.flow_from_directory(
    '/content/Dataset1',
    target_size=(224, 224),
    batch_size=32,
    subset="training"
)

val_data = datagen.flow_from_directory(
    '/content/Dataset1',
    target_size=(224, 224),
    batch_size=32,
    subset="validation"
)


Found 19999 images belonging to 1 classes.
Found 4999 images belonging to 1 classes.


In [ ]:

# Load Pre-trained Model
base_model = MobileNetV2(input_shape=(224, 224, 3), include_top=False, weights="imagenet")
base_model.trainable = False  # Freeze base model

# Custom Layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.2)(x)
output = Dense(train_data.num_classes, activation="softmax")(x)

model = Model(inputs=base_model.input, outputs=output)

# Compile and Train
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.fit(train_data, validation_data=val_data, epochs=5)

# Save Model
model_save_path = "/content/drive/MyDrive/cats_dogs_transfer_learning_model.h5"
model.save(model_save_path)

print(f"Model saved to {model_save_path}")

Epoch 1/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1509s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 2/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1516s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 3/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1475s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 4/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1548s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00
Epoch 5/5
625/625 ━━━━━━━━━━━━━━━━━━━━ 1513s 2s/step - accuracy: 1.0000 - loss: 0.0000e+00 - val_accuracy: 1.0000 - val_loss: 0.0000e+00


Model saved to /content/drive/MyDrive/cats_dogs_transfer_learning_model.h5


In [48]:
import requests

# Define the URL and destination file path
url = "https://github.com/AlexeyAB/darknet/releases/download/yolov4/yolov4.weights"
weights_path = "/content/drive/MyDrive/yolo/yolov4.weights"

# Create directory if it doesn't exist
import os
os.makedirs(os.path.dirname(weights_path), exist_ok=True)

# Download the weights file
response = requests.get(url, stream=True)
with open(weights_path, "wb") as f:
    for chunk in response.iter_content(chunk_size=1024):
        if chunk:
            f.write(chunk)

print(f"YOLOv4 weights saved at: {weights_path}")


YOLOv4 weights saved at: /content/drive/MyDrive/yolo/yolov4.weights


In [58]:
# Load YOLOv4 Model

# !pip install yolov4
# !git clone https://github.com/theAIGuysCode/yolov4-deepsort
# %cd yolov4-deepsort/
# !sed -i 's/tensorflow-gpu==2.3.0rc0/tensorflow-gpu==2.9.1/g' requirements-gpu.txt
# !pip install deep-sort-realtime



from deep_sort_realtime.deepsort_tracker import DeepSort
from yolov4.tf import YOLOv4
import cv2

yolo = YOLOv4()
yolo.config.parse_names("/content/drive/MyDrive/yolo/coco.names")
yolo.config.parse_cfg("/content/drive/MyDrive/yolo/yolov4.cfg")
yolo.make_model()
yolo.load_weights("/content/drive/MyDrive/yolo/yolov4.weights", weights_type="yolo")



In [50]:
# Initialize DeepSORT
deep_sort = DeepSort("/content/drive/MyDrive/deepsort/mars-small128.pb")


/usr/local/lib/python3.10/dist-packages/deep_sort_realtime/embedder/embedder_pytorch.py:53: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.model.load_state_dict(torch.lo

In [55]:
# Load Test Video
video_path = "/content/drive/MyDrive/test_video.mp4"
cap = cv2.VideoCapture(video_path)
out = None

In [ ]:

# Define Output Writer
output_path = "/content/drive/MyDrive/tracking_output.mp4"
frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = int(cap.get(cv2.CAP_PROP_FPS))
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_path, fourcc, fps, (frame_width, frame_height))

while True:
    ret, frame = cap.read()
    if not ret:
        break

    bboxes, scores, classes, _ = yolo.predict(frame, prob_thresh=0.5)

    # Convert to DeepSORT Format
    detections = []
    for bbox, score, cls in zip(bboxes, scores, classes):
        x_min, y_min, x_max, y_max = bbox
        detections.append((x_min, y_min, x_max - x_min, y_max - y_min, score))

    # Update Tracker
    tracks = deep_sort.update_tracks(detections, frame=frame)

    # Draw Bounding Boxes and IDs
    for track in tracks:
        if not track.is_confirmed() or track.time_since_update > 1:
            continue
        bbox = track.to_tlbr()  # Get the bounding box in (x_min, y_min, x_max, y_max)
        track_id = track.track_id
        cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)
        cv2.putText(frame, f"ID: {track_id}", (int(bbox[0]), int(bbox[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Write Frame to Output
    out.write(frame)

    # Display Frame
    cv2.imshow('Tracking', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
out.release()
cv2.destroyAllWindows()

print(f"Tracking output saved to {output_path}")
